# Supermarket Ordering, Invoicing, and Sales

Joel Day, Nicholas Lee, and Christine Vu

Shiley-Marcos School of Engineering, University of San Diego

ADS 507: Practical Data Engineering

Professor Jonathan Sixt

February 8, 2023

***

## Data Description

### Invoices.csv

| Variable | Description  |
| --- | --- |
| Order Id | The order identification number |
| Date | The date the order was placed |
| Meal Id | The meal identification number |
| Company Id | The company identification number |
| Date of Meal | The date the meal was served |
| Participants | The number of people who participated in the meal |
| Meal Price | The cost of the meal |
| Type of Meal | The type of meal that was ordered |

### OrderLeads.csv

| Variable | Description  |
| --- | --- |
| Order Id | The order identification number |
| Company Id | The company identification number |
| Company Name | The name of the company associated with the order |
| Date | The date the order was placed |
| Order Value | The total value of the order |
| Converted | Whether or not the order was converted into a sale |

### SalesTeam.csv

| Variable | Description  |
| --- | --- |
| Sales Rep | The name of the sales representative |
| Sales Rep Id | The sales representative identification number |
| Company Name | The name of the company associated with the order |
| Company Id | The company identification number |

***

## Data Importing and Pre-processing

In [44]:
# Packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

import pymysql
from sqlalchemy import create_engine
import requests
import io
import os

import warnings
warnings.filterwarnings("ignore")

Import in CSV files

In [2]:
# Function to Pull Raw CSV from GitHub and Convert to Pandas Dataframe Object

def github_to_pandas(raw_git_url):
    # Pull Raw CSV File from GitHub
    file_name = str(raw_git_url)
    pull_file = requests.get(file_name).content

    # Convert Raw CSV to Pandas Dataframe
    csv_df = pd.read_csv(io.StringIO(pull_file.decode('utf-8')))

    return csv_df

In [3]:
# Pull CSV files from GitHub and Convert to Pandas Dataframe
invoice_df = github_to_pandas(
    "https://raw.githubusercontent.com/nlee98/ADS-507-Data-Engineering/main/Invoices.csv")

orderleads_df = github_to_pandas(
    "https://raw.githubusercontent.com/nlee98/ADS-507-Data-Engineering/main/OrderLeads.csv")

salesteam_df = github_to_pandas(
    "https://raw.githubusercontent.com/nlee98/ADS-507-Data-Engineering/main/SalesTeam.csv")

### Data Pre-processing

In [4]:
# Find missing values
print("- Invoice Missing Values:\n", invoice_df.isnull().sum())
print("\n- Order Leads Missing Values:\n", orderleads_df.isnull().sum())
print("\n- Sales Team Missing Values:\n", salesteam_df.isnull().sum())

- Invoice Missing Values:
 Order Id        0
Date            0
Meal Id         0
Company Id      0
Date of Meal    0
Participants    0
Meal Price      0
Type of Meal    0
dtype: int64

- Order Leads Missing Values:
 Order Id        0
Company Id      0
Company Name    0
Date            0
Order Value     0
Converted       0
dtype: int64

- Sales Team Missing Values:
 Sales Rep       0
Sales Rep Id    0
Company Name    0
Company Id      0
dtype: int64


In [5]:
# Data types of all columns
print("- Invoice Data Types:\n", invoice_df.dtypes)
print("\n- Order Leads Data Types:\n", orderleads_df.dtypes)
print("\n- Sales Team Data Types:\n", salesteam_df.dtypes)

- Invoice Data Types:
 Order Id        object
Date            object
Meal Id         object
Company Id      object
Date of Meal    object
Participants    object
Meal Price       int64
Type of Meal    object
dtype: object

- Order Leads Data Types:
 Order Id        object
Company Id      object
Company Name    object
Date            object
Order Value      int64
Converted        int64
dtype: object

- Sales Team Data Types:
 Sales Rep       object
Sales Rep Id    object
Company Name    object
Company Id      object
dtype: object


In [6]:
# Duplicated data
print("- Invoice Duplicated Values:", invoice_df.duplicated().sum())
print("- Order Leads Duplicated Values:", orderleads_df.duplicated().sum())
print("- Sales Team Duplicated Values:", salesteam_df.duplicated().sum())

- Invoice Duplicated Values: 0
- Order Leads Duplicated Values: 0
- Sales Team Duplicated Values: 0


***

## Explore CSV Files

### Invoice CSV

In [7]:
invoice_df.head(3)

,Order Id,Date,Meal Id,Company Id,Date of Meal,Participants,Meal Price,Type of Meal
0,839FKFW2LLX4LMBB,27-05-2016,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00+02:00,['David Bishop'],469,Breakfast
1,97OX39BGVMHODLJM,27-09-2018,J0MMOOPP709DIDIE,LJKS5NK6788CYMUU,2018-10-01 20:00:00+02:00,['David Bishop'],22,Dinner
2,041ORQM5OIHTIU6L,24-08-2014,E4UJLQNCI16UX5CS,LJKS5NK6788CYMUU,2014-08-23 14:00:00+02:00,['Karen Stansell'],314,Lunch


#### Transformations
* Add Underscores to each column name
* Transform Date and Date of Meal to date/datetime data types
* Time of day column
* Number of participants column

In [8]:
# Replace spaces with underscores in all dataframe column names
invoice_df.columns = invoice_df.columns.str.replace(" ", "_")
orderleads_df.columns = orderleads_df.columns.str.replace(" ", "_")
salesteam_df.columns = salesteam_df.columns.str.replace(" ", "_")

In [9]:
# Date to Date ("d-m-Y")
invoice_df["Date"] = pd.to_datetime(
    invoice_df["Date"], format='%d-%m-%Y')

In [10]:
# Drop "+HH:MM:SS" to make all uniform to UTC timezone
invoice_df["Date_of_Meal"] = invoice_df["Date_of_Meal"].apply(
    lambda x: x.split("+")[0]
)

# Convert Date_of_Meal to Datetime format
invoice_df["Date_of_Meal"] = pd.to_datetime(
    invoice_df["Date_of_Meal"],
    format = "%Y-%m-%d %H:%M:%S"
)

In [11]:
# Convert Date_of_Meal to Datetime format
invoice_df["Date_of_Meal"] = pd.to_datetime(
    invoice_df["Date_of_Meal"],
    format = "%Y-%m-%d %H:%M:%S"
)

In [12]:
# Function defining hour of the day with the time of day
def time_of_day(x):
    day_hour = x.hour
    if (day_hour >= 5) and (day_hour <= 8): # 5am - 8am
        return "Early Morning"
    elif (day_hour > 8) and (day_hour <= 12): # 9am - 12pm
        return "Late Morning"
    elif (day_hour > 12) and (day_hour <= 15): # 1pm - 3pm
        return "Early Afternoon"
    elif (day_hour > 15) and (day_hour <= 19): # 4pm - 7pm
        return "Evening"
    elif (day_hour > 19) and (day_hour <= 23): # 8pm - 11pm
        return "Night"
    else: # 12am - 4am
        return "Late Night"

In [13]:
# Apply time_of_day function to Date_of_Meal column

invoice_df["Part_of_Day"] = invoice_df["Date_of_Meal"].apply(time_of_day)

In [14]:
# Add a field to count the number of participants
invoice_df['Number_of_Participants'] = invoice_df['Participants'].apply(lambda x: x.count("'")/2)
invoice_df['Number_of_Participants'] = invoice_df['Number_of_Participants'].astype(int)

invoice_df.head(5)

,Order_Id,Date,Meal_Id,Company_Id,Date_of_Meal,Participants,Meal_Price,Type_of_Meal,Part_of_Day,Number_of_Participants
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00,['David Bishop'],469,Breakfast,Early Morning,1
1,97OX39BGVMHODLJM,2018-09-27,J0MMOOPP709DIDIE,LJKS5NK6788CYMUU,2018-10-01 20:00:00,['David Bishop'],22,Dinner,Night,1
2,041ORQM5OIHTIU6L,2014-08-24,E4UJLQNCI16UX5CS,LJKS5NK6788CYMUU,2014-08-23 14:00:00,['Karen Stansell'],314,Lunch,Early Afternoon,1
3,YT796QI18WNGZ7ZJ,2014-04-12,C9SDFHF7553BE247,LJKS5NK6788CYMUU,2014-04-07 21:00:00,['Addie Patino'],438,Dinner,Night,1
4,6YLROQT27B6HRF4E,2015-07-28,48EQXS6IHYNZDDZ5,LJKS5NK6788CYMUU,2015-07-27 14:00:00,['Addie Patino' 'Susan Guerrero'],690,Lunch,Early Afternoon,2


### Customer-Order Table
Connect the customer id to each order id the customer placed. This table will link the customer information to the invoice information.

In [15]:
# Find all the occurrences of customer names then explode to convert values in lists to rows
cust = invoice_df['Participants'].str.findall(r"'(.*?)'").explode()

# Join with order id 
cust_order_df = invoice_df[['Order_Id']].join(cust)

# Factorize to encode the unique values in participants
cust_order_df['Customer_Id'] = cust_order_df['Participants'].factorize()[0] + 1

cust_order_df.head(9)

,Order_Id,Participants,Customer_Id
0,839FKFW2LLX4LMBB,David Bishop,1
1,97OX39BGVMHODLJM,David Bishop,1
2,041ORQM5OIHTIU6L,Karen Stansell,2
3,YT796QI18WNGZ7ZJ,Addie Patino,3
4,6YLROQT27B6HRF4E,Addie Patino,3
4,6YLROQT27B6HRF4E,Susan Guerrero,4
5,AT0R4DFYYAFOC88Q,David Bishop,1
5,AT0R4DFYYAFOC88Q,Susan Guerrero,4
5,AT0R4DFYYAFOC88Q,Karen Stansell,2


### Order Leads CSV
* Converted Column - Whether or not a order was converted into a sale

In [16]:
orderleads_df.head(3)

,Order_Id,Company_Id,Company_Name,Date,Order_Value,Converted
0,80EYLOKP9E762WKG,LJKS5NK6788CYMUU,Chimera-Chasing Casbah,18-02-2017,4875,1
1,TLEXR1HZWTUTBHPB,LJKS5NK6788CYMUU,Chimera-Chasing Casbah,30-07-2015,8425,0
2,839FKFW2LLX4LMBB,LJKS5NK6788CYMUU,Chimera-Chasing Casbah,27-05-2016,4837,0


### Sales Team CSV

In [17]:
salesteam_df.head(3)

,Sales_Rep,Sales_Rep_Id,Company_Name,Company_Id
0,Jessie Mcallister,97UNNAT790E0WM4N,Chimera-Chasing Casbah,LJKS5NK6788CYMUU
1,Jessie Mcallister,97UNNAT790E0WM4N,Tangential Sheds,36MFTZOYMTAJP1RK
2,Jessie Mcallister,97UNNAT790E0WM4N,Two-Mile Grab,H3JRC7XX7WJAD4ZO


***

## Connection to MySQL Server

In [18]:
# Manually Login to MySQL
mysql_username = str(input("Enter MySQL Username: "))
mysql_password = str(input("Enter MySQL Password: "))

mysql_conn = pymysql.connect(
    host = "localhost",
    port = int(3306),
    user = mysql_username,
    passwd = mysql_password
)

### Create Supermarket Database - if it does not already exist

In [19]:
# Create ADS-507_Supermarket MySQL Database
mysql_conn.cursor().execute(
    """
    CREATE DATABASE IF NOT EXISTS ADS_507_Supermarket;
    """
)

# Navigate to Supermarket Database
mysql_conn.select_db("ADS_507_Supermarket")

In [48]:
create_invoice_table = """
CREATE TABLE IF NOT EXISTS invoice(
    Order_id TINYINT,
    Date DATE, 
    Meal_id TINYINT, 
    Company_id TINYINT,
    Date_of_Meal DATETIME,
    Participants VARCHAR(255),
    Meal_Price SMALLINT,
    Type_of_Meal ENUM('Breakfast', 'Lunch', 'Dinner'),
    Part_of_Day ENUM('Early Morning', 'Late Morning', 'Early Afternoon', 'Night', 'Late Night'),
    Number_of_Participants TINYINT,
    PRIMARY KEY (Order_id)
   )
"""


mysql_conn.cursor().execute(create_invoice_table);

In [52]:
create_order_table = """
    CREATE TABLE IF NOT EXISTS orders(
        Order_id TINYINT,
        Company_id TINYINT,
        Company_Name VARCHAR(255),
        Date DATE,
        Order_Value SMALLINT,
        Converted TINYINT,
        PRIMARY KEY (Order_id, Company_id)
    )
"""

mysql_conn.cursor().execute(create_order_table);

In [56]:
create_salesteam_table = """
    CREATE TABLE IF NOT EXISTS salesteam(
        Sales_Rep VARCHAR(255),
        Sales_Rep_id TINYINT,
        Company_Name VARCHAR(255),
        Company_id TINYINT,
        PRIMARY KEY (Sales_Rep_id)
    )
"""

mysql_conn.cursor().execute(create_salesteam_table);

In [45]:
# Create Engine to write to SQL table
engine = create_engine(
    "mysql+pymysql://{user}:{pw}@{host}/{db}".format(
        host = "localhost", 
        db = "ADS_507_Supermarket", 
        user = mysql_username, 
        pw = mysql_password))

## Upload dataframes as tables into MySQL
* Invoice
* Orders
* Sales Lead
* Customer (cust_order_df)

In [57]:
# If CSV is updated, the whole table is re-written
def insert_table(df_name, sql_table):
    df_name.to_sql(sql_table, 
                  engine, 
                  index = False, 
                  if_exists = 'replace')
    
#  Convert Pandas dataframes to SQL tables
insert_table(invoice_df, "invoice")
insert_table(orderleads_df, "orders")
insert_table(salesteam_df, "salesteam")